In [10]:
import os
from os import listdir
from pathlib import *
from pathlib import Path
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib

matplotlib.use('TKAgg')
import shutil
from matplotlib import pyplot as plt
from pixelmatch.contrib.PIL import pixelmatch as pm2

In [11]:
matplotlib.get_backend()

'TkAgg'

# Image lines segmentation

Find the edges of the image (i.e. the edges of the single lines).


In [12]:
def get_line(img, dr):
    img = cv.imread(img)
    grey = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    contours, hierarchy = cv.findContours(grey, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    line_num = len(contours)
    print("Number of line elements found: " + str(line_num))

    for x in range(len(contours)):
        #selecting line x of those detected
        bb = contours[x]
        #creating a stencil of the original image
        #1. preparing the black background image;
        stencil = np.zeros(img.shape).astype(img.dtype)
        #2. set into an array the actual contours of the 'line element';
        cont = [np.array(bb)]
        #3. fill the contours with white color (i.e. [255, 255, 255]);
        line = cv.fillPoly(stencil, cont, color=(255, 255, 255))

        #_._._Saving step
        dr = './' + str(dr) + '/'
        filename = dr + 'line' + str(x) + '.png'
        cv.imwrite(filename, line)

    return (line_num)

# Intersection over Union

In [13]:
#Defining a function to calculate IoU on the fly
def IoU(l1, l2):
    intersection = np.logical_and(l1, l2)
    union = np.logical_or(l1, l2)
    iou_score = np.sum(intersection) / np.sum(union)
    return (iou_score)

# Recall and Percision

Procedure: https://www.jeremyjordan.me/evaluating-image-segmentation-models/

In [14]:
def recall(gt, prd):

    #setting variables
    score = []
    TP = 0
    FN = 0
    threshold = .45
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    xtr_ln = abs(pn - gtn)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(gt)):
        if (img.endswith(".png")):
            image1 = cv.imread(gt + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(prd)):
                if (img2.endswith(".png")):
                    image2 = Image.open(prd + img2)

                    #print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FN += 1
            score = []

    Rec = TP / (TP + FN)

    #out = "Recall score: " + str(Rec)

    return (Rec)

In [15]:
def precision(gt, prd):

    #setting variables
    score = []
    TP = 0
    FP = 0
    threshold = .45
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(prd)):
        if (img.endswith(".png")):
            image1 = cv.imread(prd + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(gt)):
                if (img2.endswith(".png")):
                    image2 = Image.open(gt + img2)

                    #print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FP += 1
            score = []

    Precision = TP / (TP + FP)

    #out = "Precision score: " + str(Precision)

    return (Precision)

In [ ]:
def precision(gt, prd):

    #setting variables
    score = []
    TP = 0
    FP = 0
    threshold = .45
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(prd)):
        if (img.endswith(".png")):
            image1 = cv.imread(prd + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(gt)):
                if (img2.endswith(".png")):
                    image2 = Image.open(gt + img2)

                    #print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FP += 1
            score = []

    Precision = TP / (TP + FP)

    #out = "Precision score: " + str(Precision)

    return (Precision)

In [16]:
def get_fname(dir):
    filename = dir.split('/')
    filename = filename[-1]
    return(filename)

# Main

In [23]:
import os
import glob

def main():

    n=0
    rec=0
    prec=0
    iou=0
    direc= os.getcwd()

    for img in glob.glob(direc + '/gt/*.png'):
        store_r=0
        store_p=0
        for img2 in glob.glob(direc + '/output/*.png'):
            if get_fname(img) == get_fname(img2):
              #_._.create working folders
              os.mkdir(direc + '/linegt')
              os.mkdir(direc + '/lineprd')
              get_line(img, 'linegt')
              get_line(img2, 'lineprd')
              lin_gt = direc + '/linegt/'
              lin_prd = direc + '/lineprd/'
              store_r = recall(lin_gt, lin_prd)
              rec = rec + store_r
              print('Recall: ' + str(store_r))
              store_p = precision(lin_gt, lin_prd)
              prec = prec + store_p
              print('Precision: ' + str(store_p))
              iou += IoU(cv.imread(img), cv.imread(img2))
              print('IoU: ' + str(iou))
              n+=1
              #_._.remove working folders
              shutil.rmtree(direc + '/linegt')
              shutil.rmtree(direc + '/lineprd')

    rec_mean = rec/n
    prec_mean = prec/n
    iou_mean = iou/n
    reca='\n\nRecall mean over '+ str(n) + ' documents: '+ str(rec_mean)
    preci= '\n\nPrecision mean over '+ str(n) + ' documents: '+ str(prec_mean)
    i_o_u= '\n\nIoU mean over '+ str(n) + ' documents: '+ str(iou_mean)
    out = reca + preci + i_o_u
    print(out)
    return ()

In [ ]:
if __name__ == "__main__":
    main()